In [251]:
import pandas as pd
import numpy as np
import random
import os
import datetime
import string

# import data
data = pd.read_csv(
    "/home/bluepi/Downloads/Update/product_info/main table/product_info.csv")
new_product = pd.read_csv(
    "/home/bluepi/Downloads/Update/product_info/main table/new_added.csv")

In [243]:
# Products which have been deleted
deleted_product = []

# Products which have been Inserted
inserted_product = []

# Getting list of all products present in main product table
data_pid = data['p_id'].tolist()

# *****************************************************************************************

# This for-loop is created date wise directories in incremental order
# If you want data of past 10 days
for i in range(10, 0, -1):

    # Get the current date
    current_date = datetime.datetime.today()

    # datetime.timedelta(days=1) ---> in order to get the previous date
    # strftime ('%d-%m-%Y')  ---> To get the date in time desired format
    date_folder_raw = current_date - datetime.timedelta(days=i)


    date_folder = date_folder_raw.strftime('%d-%m-%Y')

#    print("folder name ---> "+date_folder,end='\n')

    # Parent Directory path
    parent_dir = "/home/bluepi/Downloads/Update/product_info"
    # os.path.join --> join one or more path components
    path = os.path.join(parent_dir, date_folder)

    # Creating Directory
    try:
        os.mkdir(path)
    except Exception:
        if len(os.listdir(path)) == 0:
            print(date_folder + " --> directory already present and empty")
        else:
            print(date_folder + " --> directory already present and not empty")
            continue

# *****************************************************************************************
    limit = datetime.timedelta(hours=24)
    day_hour_begin = datetime.timedelta(hours=2)
    day_hour = day_hour_begin
    
# This for loop is to create files inside the folders
# We are updating our product 4 times a day only
    for j in range(4):

        # This section is for updating existing products :-

        # Next file created in the interval of of 4 hours
        day_hour += datetime.timedelta(hours=4) + day_hour_begin
        
        # Since we want our day_hour variable to within a range of 24 hours
        if day_hour > limit :
            day_hour = day_hour_begin

        # Generate random values
        value = random.randint(1, 10)

        # This is used to sample out random number of rows
        data_random_subset = data.sample(n=value)

# We dont want our deleted products to be included in as a updated product
#        print("deleted_product",end = '\n')
#        print(deleted_product,end = '\n')

################################################################

#        print("data_random_subset before deleting--->")
#        print(data_random_subset['p_id'].values,end='\n')
        for k in deleted_product:
            if (k in data_random_subset['p_id'].values):
                # Get names of indexes for which p_id has value equal to i
                indexNames = data_random_subset[data_random_subset['p_id'] == k].index

                # Print it
#                print(data_random_subset[data_random_subset['p_id'] == k],end = ' ')

                # Delete these row indexes from dataFrame
                data_random_subset.drop(indexNames, inplace=True)

#        print("\n data_random_subset after deleting--->")
#        print(data_random_subset['p_id'].values,end='\n')

#################################################################

        # Drop the "existing or old price" price column
        data_random_subset = data_random_subset.drop(['price', 'day', 'time'],
                                                     axis=1)

        # Add new price column with new prices
        # size --> size of array which we want
        data_random_subset['price'] = np.random.randint(
            1, 1500, size=data_random_subset.shape[0])

        # To create date_timestamp
        # here we used list comprehensions
        # datetime.datetime.today() ---> get the today's date

        # datetime.timedelta(minutes= ...,seconds=...) ---> used for calculating differences in dates & date manipulations
        data_random_subset['Date_timestamp'] = [
            (date_folder_raw - datetime.timedelta(days=1) + day_hour +
             datetime.timedelta(minutes=random.randint(10, 100),
                                seconds=random.randint(
                                    10, 100))).strftime("%Y-%m-%d %H:%M:%S")
            for i in range(data_random_subset.shape[0])
        ]

        # Adding record type data
        data_random_subset['record_type'] = 'U'

        # *****************************************************************************************

# This section is for inserting New Products :-

        # Generate random values
        value_2 = random.randint(0, 5)

        # new products picked at random
        new_data_random_subset = new_product.sample(n=value_2)

        # Updating new product file
        new_product[~new_product['p_id'].isin(
            new_data_random_subset['p_id'].values)]

        # Adding the date timestamp
        new_data_random_subset['Date_timestamp'] = [
            (date_folder_raw - datetime.timedelta(days=1) + day_hour +
             datetime.timedelta(minutes=random.randint(10, 100),
                                seconds=random.randint(
                                    10, 100))).strftime("%Y-%m-%d %H:%M:%S")
            for i in range(new_data_random_subset.shape[0])
        ]

        # Adding the record type
        new_data_random_subset['record_type'] = 'I'

        # Concating newly added and updated products
        final_day_update = pd.concat(
            [data_random_subset, new_data_random_subset], sort=False)

        # *****************************************************************************************

# # This section is for Deleting the Products :-

        # Picking up the pid from main table which we want to delete
        to_be_deleted = random.choices(data_pid, k=random.randint(0, 4))

#        print(" to be deleted products--> ");print(to_be_deleted,end = '\n')

        # Check wheather the product is already deleted or not
        for i in to_be_deleted:
            # If pid is already deleted
            if i in deleted_product:
                # REMOVE IT.
                to_be_deleted.remove(i)
            else:
                # If not append it to the deleted products list
                deleted_product.append(i)

# Creating a dataframe of the products which have to be deleted
        deleted_that_day = data[data['p_id'].isin(to_be_deleted)]
        # Drop the "existing or old price" price column
        deleted_that_day = deleted_that_day.drop(['day', 'time'], axis=1)

        # Adding the date timestamp
        deleted_that_day['Date_timestamp'] = [
            (date_folder_raw - datetime.timedelta(days=1) + day_hour +
             datetime.timedelta(minutes=random.randint(10, 100),
                                seconds=random.randint(
                                    10, 100))).strftime("%Y-%m-%d %H:%M:%S")
            for i in range(deleted_that_day.shape[0])
        ]

        # Adding the record type
        deleted_that_day['record_type'] = 'D'

        # Concating newly added and updated products
        final_final_day_update = pd.concat(
            [final_day_update, deleted_that_day], sort=False)

        # *****************************************************************************************

        # Create the file name as it would be random
        file_name = str(''.join(
            random.choices(string.ascii_uppercase + string.digits, k=10)))
        # File path
        path_csv = path + '/' + file_name + '.csv'
        # Exporting the csv file
#        print('\n\n')
        final_final_day_update.to_csv(path_csv, index=False)

## TO DO :-
### - delete the products which are already inserted in the files.
### - Change name of duplicate "p_name"